In [ ]:
import cv2
import dlib
import math
import tensorflow as tf
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from PIL import Image

image_paths = '/content/drive/MyDrive/SCUT-FBP5500_v2/Images'


images = []


for file_name in os.listdir(image_paths):
    file_path = os.path.join(image_paths, file_name)


    if os.path.isfile(file_path) and file_name.lower().endswith(('.jpg', '.jpeg', '.png')):

        image = Image.open(file_path)


        images.append((image,file_name))



In [ ]:
pip install dlib

In [ ]:
import dlib
import pandas as pd
from PIL import Image
import numpy as np
import math

In [ ]:
def detect_landmarks(image):

  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  faces = detector(gray)

  for face in faces:
    landmarks = predictor(gray,face)

    points = np.zeros((68,2),dtype=int)
    for i in range(0,68):
      points[i] = (landmarks.part(i).x, landmarks.part(i).y)

    return points

In [ ]:
def calculate_golden_ratios(landmarks):
    # Calculating Euclidean distance between two landmarks
    under_eyes = np.linalg.norm(landmarks[41]-landmarks[46])
    interocular = np.linalg.norm(landmarks[39]-landmarks[42])
    nose_width = np.linalg.norm(landmarks[31]-landmarks[35])
    mouth_width = np.linalg.norm(landmarks[48]-landmarks[54])
    upper_lip_jaw = np.linalg.norm(landmarks[51]-landmarks[8])
    lip_height = np.linalg.norm(landmarks[51]-landmarks[57])
    nose_mouth_height = np.linalg.norm(landmarks[33]-landmarks[51])
    eyebrows_nose = np.linalg.norm((np.linalg.norm(landmarks[21]-landmarks[22]))-landmarks[33])/2
    nose_jaw = np.linalg.norm(landmarks[33]-landmarks[9])
    upper_lip_height = np.linalg.norm(landmarks[51]-landmarks[56])/2



    # Calculate the golden ratios
    ratio1 = under_eyes / interocular
    ratio2 = under_eyes / nose_width
    ratio3 = mouth_width / interocular
    ratio4 = upper_lip_jaw/interocular
    ratio5 = upper_lip_jaw / nose_width
    ratio6 = interocular / lip_height
    ratio7 = nose_width / interocular
    ratio8 = nose_width / upper_lip_height
    ratio9 = interocular/nose_mouth_height
    ratio10 = eyebrows_nose / nose_jaw
    ratio11 = interocular / nose_width

    return ratio1, ratio2, ratio3, ratio4, ratio5, ratio6, ratio7, ratio8, ratio9, ratio10, ratio11

In [ ]:
golden_ratios = []
counter = 1

for image, name in images:

    image_array = np.array(image)

    # Detect landmarks
    landmarks = detect_landmarks(image_array)

    # Check if landmarks were found
    if landmarks is not None:
        # Calculate golden ratios
        print("Calculated Golden ratios for ",counter," number of images")
        counter = counter + 1
        ratio1, ratio2, ratio3, ratio4, ratio5, ratio6, ratio7, ratio8, ratio9, ratio10, ratio11 = calculate_golden_ratios(landmarks)

        # Store the ratios along with the image name
        golden_ratios.append({'Image Name': name, 'UE/IO': ratio1, 'UE/NW': ratio2, 'MW/IO': ratio3, 'ULJ/IO': ratio4, 'ULJ/NW': ratio5, 'IO/LH': ratio6, 'NW/IO': ratio7, 'NW/ULH': ratio8, 'IO/NMH': ratio9, 'EBN/NJ': ratio10, 'IO/NW': ratio11})
    else:
        print(f"No landmarks found for image: {name}")


df = pd.DataFrame(golden_ratios)



In [ ]:
df

,Image Name,UE/IO,UE/NW,MW/IO,ULJ/IO,ULJ/NW,IO/LH,NW/IO,NW/ULH,IO/NMH,EBN/NJ,IO/NW
0,AM218.jpg,1.981239,2.836753,1.339132,1.344277,1.924745,2.119258,0.698418,2.810133,3.296268,1.369064,1.431807
1,AM1948.jpg,1.833635,2.929858,1.270414,1.030303,1.646262,3.002603,0.625844,3.437585,1.994821,1.541130,1.597841
2,AM1946.jpg,1.933288,2.395209,1.465257,1.564513,1.938322,1.566378,0.807148,2.495686,2.343096,1.211412,1.238930
3,AM214.jpg,1.882761,2.341275,1.431507,1.531422,1.904373,2.315791,0.804161,3.459990,2.684211,1.335612,1.243533
4,AM1939.jpg,1.937528,2.324811,1.584223,1.333695,1.600281,3.693109,0.833413,4.915896,3.172618,1.518923,1.199885
...,...,...,...,...,...,...,...,...,...,...,...,...
5480,CF710.jpg,1.924013,2.483051,1.735093,1.370213,1.768340,1.550767,0.774858,2.191186,6.519202,1.457473,1.290558
5481,CF438.jpg,1.888889,2.644444,1.650870,1.587381,2.222333,1.399654,0.714286,1.922305,3.607370,1.023950,1.400000
5482,CF711.jpg,2.203964,3.361684,1.646898,1.818176,2.773245,1.685601,0.655613,2.082643,2.113201,0.896155,1.525290
5483,CF444.jpg,2.017300,2.705405,1.796432,1.680804,2.254128,1.282793,0.745656,1.855203,3.418269,0.875941,1.341102


In [ ]:
df.to_csv('/content/drive/MyDrive/golden_ratios.csv', index=False)
